In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import pickle
from matplotlib import pyplot as plt

## PREPARATION

### Functions

In [2]:
import time
from collections import Counter
from sklearn.model_selection import KFold, cross_val_score, RandomizedSearchCV, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, precision_score, recall_score, make_scorer
from sklearn.neighbors import KDTree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials, space_eval
from catboost import CatBoostClassifier
import xgboost as xgb

In [3]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [4]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    clf = model
    clf.fit(X_train, y_train)
    print_result(clf, X_train, X_test, y_train, y_test)
    return(clf.predict(X_test))

In [5]:
def print_result(clf, X_train, X_test, y_train, y_test):
    print('Accuracy Test :', f'{accuracy_score(clf.predict(X_test), y_test):.4f}', 
          '| F1 Test :', f'{f1_score(clf.predict(X_test), y_test, pos_label="Bad"):.4f}',
          '| Precision Test :', f'{precision_score(clf.predict(X_test), y_test, pos_label="Bad"):.4f}', 
          '| Recall Test :', f'{recall_score(clf.predict(X_test), y_test, pos_label="Bad"):.4f}', 
          '| H Test :', f'{H_score(clf.predict(X_test), y_test):.4f}')
    
    print('Accuracy Train:', f'{accuracy_score(clf.predict(X_train), y_train):.4f}', 
          '| F1 Train:', f'{f1_score(clf.predict(X_train), y_train, pos_label="Bad"):.4f}',
          '| Precision Train:', f'{precision_score(clf.predict(X_train), y_train, pos_label="Bad"):.4f}', 
          '| Recall Train:', f'{recall_score(clf.predict(X_train), y_train, pos_label="Bad"):.4f}', 
          '| H Train:', f'{H_score(clf.predict(X_train), y_train):.4f}')

In [6]:
def H_score(X_train, y_train):
    acc = accuracy_score(X_train, y_train)
    f1 = f1_score(X_train, y_train, pos_label = "Bad")
    return(2 / ((1/(acc+0.0000001))+(1/(f1+0.0000001))))

In [7]:
def bayesian(space, X, y, modelo, nevals):
    
    f1 = make_scorer(f1_score, pos_label = "Bad")
    H = make_scorer(H_score, greater_is_better=True) 
    
    def objective(space):        
        global best_score
        model = modelo(**space, random_state = 1)   
        cv =  StratifiedKFold(n_splits = 5, random_state = 1)
        score = -cross_val_score(model, X, y, cv = cv, scoring = H, verbose = False).mean()
        if (score < best_score):
            best_score = score
        return score

    start = time.time()
    rstate = np.random.RandomState(1)
    best = fmin(objective, space = space, algo = tpe.suggest, max_evals = nevals,trials = Trials(), rstate = rstate)

    print("Hyperopt search took %.2f seconds" % ((time.time() - start)))
    print("Best score: %.4f " % (-best_score))
    print("Best space: ", space_eval(params, best))
    return(space_eval(params, best))

### Create Data Frame

In [8]:
df_balanced = pickle.load(open('./sav/df_balanced.sav', 'rb'))
df_subset = pickle.load(open('./sav/df_subset.sav', 'rb'))
df_stacking = df_balanced.drop(df_subset.index).sample(n=10000, random_state=11)

In [9]:
y = df_subset['Category']
X = df_subset[df_subset.columns[1:-1]]

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

In [11]:
X_test.shape, y_test.shape, X_train.shape, y_train.shape

((2000, 38), (2000,), (8000, 38), (8000,))

In [12]:
y = df_stacking['Category']
X = df_stacking[df_stacking.columns[1:-1]]

### Add Predictions

In [13]:
models = pickle.load(open('./sav/model_H.sav', 'rb'))

In [14]:
pred_knn = [Counter([y.iloc[k] for k in x]).most_common(1)[0][0] for x in models[0].query(X, k = 100)[1]]
pred_log = models[1].predict(X)
pred_svm = models[2].predict(X)
pred_tree = models[3].predict(X)
pred_rf = models[4].predict(X)
pred_gbt = models[5].predict(X)
pred_xgb = models[6].predict(X)
pred_cat = models[7].predict(X)

And I append the predictions of the model to the dataset

In [15]:
X['logistic'] = pred_log
X['gbt'] = pred_gbt
X['knn'] = pred_knn
X['svm'] = pred_svm
X['tree'] = pred_tree
X['xgb'] = pred_xgb
X['rf'] = pred_rf
X['cat'] = pred_cat

In [16]:
X.iloc[:,-8:].head(5)

,logistic,gbt,knn,svm,tree,xgb,rf,cat
231987,Bad,Bad,Bad,Bad,Bad,Bad,Bad,Bad
178130,Good,Good,Good,Good,Good,Good,Good,Good
266651,Bad,Bad,Bad,Bad,Bad,Bad,Bad,Bad
428706,Bad,Bad,Bad,Bad,Bad,Bad,Bad,Bad
408969,Bad,Bad,Bad,Bad,Bad,Bad,Bad,Bad


In [17]:
X.iloc[:,-8:] = X.iloc[:,-8:].apply(lambda x: [1 if i=='Good' else 0 for i in x])

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

## MODELS

### Gradient Boosting Trees

In [19]:
params = {'learning_rate':     hp.choice('learning_rate',[0.0001, 0.00025, 0.0005, 0.00075, 0.001, 0.0025, 0.005, 
                                                          0.0075, 0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.50, 0.75, 1]), 
          'n_estimators':      hp.choice('n_estimators', range(1,400)),
          'max_depth':         hp.choice('max_depth',range(1,20)),
          'min_samples_split': hp.choice('min_samples_split',np.linspace(0.01, 1.0, 10, endpoint=True)),
          'min_samples_leaf':  hp.choice('min_samples_leaf',np.linspace(0.01, 0.5, 50, endpoint=True)), 
          'subsample':         hp.choice('subsample',[1]), 
          'max_features':      hp.choice('max_features',['sqrt'])}

best_score = 1
gbt_params = bayesian(params, X_train, y_train, GradientBoostingClassifier, 50)
pred_gbt_stck = evaluate_model(GradientBoostingClassifier(**gbt_params, random_state=22), X_train, X_test, y_train, y_test)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:51<00:00,  2.24s/trial, best loss: -0.6651058133191825]
Hyperopt search took 112.07 seconds
Best score: 0.6651 
Best space:  {'learning_rate': 0.0025, 'max_depth': 13, 'max_features': 'sqrt', 'min_samples_leaf': 0.15000000000000002, 'min_samples_split': 0.23, 'n_estimators': 60, 'subsample': 1}
Accuracy Test : 0.6500 | F1 Test : 0.6841 | Precision Test : 0.7446 | Recall Test : 0.6327 | H Test : 0.6666
Accuracy Train: 0.6506 | F1 Train: 0.6806 | Precision Train: 0.7312 | Recall Train: 0.6366 | H Train: 0.6653


### XGBoosting

In [20]:
params = {'learning_rate':    hp.choice('learning_rate',[0.0001, 0.00025, 0.0005, 0.00075, 0.001, 0.0025, 0.005, 0.0075, 
                                                         0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5, 0.75]), 
          'max_depth':        hp.choice('max_depth',range(1,20)),
          'min_child_weight': hp.choice('min_child_weight',np.linspace(0.01, 1.0, 100, endpoint=True)),
          'gamma':            hp.choice('gamma',np.linspace(0.01, 1.0, 100, endpoint=True)), 
          'colsample_bytree': hp.choice('colsample_bytree',np.linspace(0.0, 1, 101, endpoint=True)), 
          'n_estimators':     hp.choice('n_estimators', range(1,200))}

best_score = 1
xgb_params = bayesian(params, X_train, y_train, xgb.XGBClassifier, 50)
pred_xgb_stck = evaluate_model(xgb.XGBClassifier(**xgb_params, random_state=22), X_train, X_test, y_train, y_test)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [05:25<00:00,  6.51s/trial, best loss: -0.6611624326008452]
Hyperopt search took 325.63 seconds
Best score: 0.6612 
Best space:  {'colsample_bytree': 0.96, 'gamma': 0.89, 'learning_rate': 0.00025, 'max_depth': 3, 'min_child_weight': 0.01, 'n_estimators': 63}
Accuracy Test : 0.6545 | F1 Test : 0.6799 | Precision Test : 0.7210 | Recall Test : 0.6433 | H Test : 0.6670
Accuracy Train: 0.6539 | F1 Train: 0.6766 | Precision Train: 0.7110 | Recall Train: 0.6453 | H Train: 0.6650


### SVM

In [21]:
params = {"degree": hp.choice('degree', [2, 3, 4]),
          "kernel": hp.choice('kernel', ['poly']), 
          "C":      hp.choice('C', [0.0001, 0.00025, 0.0005, 0.00075, 0.001, 0.0025, 0.005, 0.0075, 
                                    0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5, 0.75])}
best_score = 1
svm_params = bayesian(params, X_train, y_train, SVC, 10)
pred_svm_stck = evaluate_model(SVC(**svm_params, random_state=22), X_train, X_test, y_train, y_test)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [02:33<00:00, 15.31s/trial, best loss: -0.6659307183147462]
Hyperopt search took 153.13 seconds
Best score: 0.6659 
Best space:  {'C': 0.1, 'degree': 3, 'kernel': 'poly'}
Accuracy Test : 0.6510 | F1 Test : 0.6804 | Precision Test : 0.7299 | Recall Test : 0.6372 | H Test : 0.6654
Accuracy Train: 0.6575 | F1 Train: 0.6826 | Precision Train: 0.7235 | Recall Train: 0.6461 | H Train: 0.6698


### Logistic Regression

In [22]:
params = {"C":   hp.choice('C',[0.0001, 0.00025, 0.0005, 0.001, 0.0025, 0.005, 0.01, 0.025, 0.05, 0.1]),
          ""
          "tol": hp.choice('tol',[0.00001, 0.000025, 0.00005, 0.0001, 0.00025, 0.0005, 0.001, 0.0025, 0.005, 0.01, 0.025, 
                                  0.05, 0.1])}

best_score = 1
log_params = bayesian(params, X_train, y_train, LogisticRegression, 50)
pred_log_stck = evaluate_model(LogisticRegression(**log_params, random_state=22), X_train, X_test, y_train, y_test)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:18<00:00,  2.74trial/s, best loss: -0.6593517069247362]
Hyperopt search took 18.23 seconds
Best score: 0.6594 
Best space:  {'C': 0.01, 'tol': 0.005}
Accuracy Test : 0.6505 | F1 Test : 0.6641 | Precision Test : 0.6788 | Recall Test : 0.6500 | H Test : 0.6572
Accuracy Train: 0.6575 | F1 Train: 0.6654 | Precision Train: 0.6688 | Recall Train: 0.6620 | H Train: 0.6614


### Random Forest

In [23]:
params = {'bootstrap':         hp.choice('bootstrap',[True, False]),
          'max_depth':         hp.choice('max_depth', range(1, 20)),
          'max_features':      hp.choice('max_features',['auto', 'sqrt']),
          'min_samples_leaf':  hp.choice('min_samples_leaf',np.linspace(0.01, 0.5, 50, endpoint=True)), 
          'min_samples_split': hp.choice('min_samples_split',np.linspace(0.01, 1.0, 100, endpoint=True)), 
          'n_estimators':      hp.choice('n_estimators',range(1,400))}

best_score = 1
rf_params = bayesian(params, X_train, y_train, RandomForestClassifier, 50)
pred_rf_stck = evaluate_model(RandomForestClassifier(**rf_params, random_state=22), X_train, X_test, y_train, y_test)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [02:09<00:00,  2.59s/trial, best loss: -0.6544190908989866]
Hyperopt search took 129.65 seconds
Best score: 0.6544 
Best space:  {'bootstrap': False, 'max_depth': 8, 'max_features': 'auto', 'min_samples_leaf': 0.01, 'min_samples_split': 0.7000000000000001, 'n_estimators': 82}
Accuracy Test : 0.6530 | F1 Test : 0.6683 | Precision Test : 0.6866 | Recall Test : 0.6508 | H Test : 0.6605
Accuracy Train: 0.6469 | F1 Train: 0.6585 | Precision Train: 0.6688 | Recall Train: 0.6486 | H Train: 0.6526


### CatBoost

In [24]:
cat_features = [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,
                38,39,40,41,42,43,44,45]
params = {'iterations':        hp.choice('iterations', range(100, 4000, 25)), 
          'learning_rate':     hp.choice('learning_rate', [0.001, 0.0025, 0.0075, 0.01, 0.025, 0.05, 0.1]),
          'l2_leaf_reg':       hp.choice('l2_leaf_reg', range(1, 10)), 
          'cat_features':      hp.choice('cat_features', [cat_features]), 
          'verbose':           hp.choice('verbose', [False])}

best_score = 1
cat_params = bayesian(params, X_train, y_train, CatBoostClassifier, 10)
pred_cat = evaluate_model(CatBoostClassifier(**cat_params, random_state=22), X_train, X_test, y_train, y_test)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [07:51<00:00, 47.16s/trial, best loss: -0.658670979832542]
Hyperopt search took 471.68 seconds
Best score: 0.6587 
Best space:  {'cat_features': (5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45), 'iterations': 1050, 'l2_leaf_reg': 2, 'learning_rate': 0.0075, 'verbose': False}
Accuracy Test : 0.6480 | F1 Test : 0.6644 | Precision Test : 0.6847 | Recall Test : 0.6454 | H Test : 0.6561
Accuracy Train: 0.6850 | F1 Train: 0.6940 | Precision Train: 0.7014 | Recall Train: 0.6866 | H Train: 0.6894


### Decision Trees

In [25]:
params = {"max_depth":        hp.choice('max_depth', range(1, 50)),
          "max_features":     hp.choice('max_features', range(1, X_train.columns.size)),
          "min_samples_leaf": hp.choice('min_samples_leaf', range(1, 200)),
          "criterion":        hp.choice('criterion', ["gini", "entropy"])}

best_score = 1
tree_params = bayesian(params, X_train, y_train, DecisionTreeClassifier, 50)
pred_tree = evaluate_model(DecisionTreeClassifier(**tree_params, random_state=22), X_train, X_test, y_train, y_test)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:13<00:00,  3.71trial/s, best loss: -0.659430472880886]
Hyperopt search took 13.53 seconds
Best score: 0.6594 
Best space:  {'criterion': 'entropy', 'max_depth': 4, 'max_features': 37, 'min_samples_leaf': 198}
Accuracy Test : 0.6510 | F1 Test : 0.6654 | Precision Test : 0.6817 | Recall Test : 0.6498 | H Test : 0.6581
Accuracy Train: 0.6564 | F1 Train: 0.6672 | Precision Train: 0.6767 | Recall Train: 0.6581 | H Train: 0.6618


### Best Models

In [27]:
gbt_params = {'learning_rate': 0.0025, 'max_depth': 13, 'max_features': 'sqrt', 'min_samples_leaf': 0.15000000000000002, 
              'min_samples_split': 0.23, 'n_estimators': 60, 'subsample': 1}
pred_gbt_stck = evaluate_model(GradientBoostingClassifier(**gbt_params, random_state=22), X_train, X_test, y_train, y_test)

Accuracy Test : 0.6500 | F1 Test : 0.6841 | Precision Test : 0.7446 | Recall Test : 0.6327 | H Test : 0.6666
Accuracy Train: 0.6506 | F1 Train: 0.6806 | Precision Train: 0.7312 | Recall Train: 0.6366 | H Train: 0.6653


In [28]:
pd.crosstab(pred_gbt_stck, y_test)

Category,Bad,Good
row_0,,
Bad,758,440
Good,260,542


### Save Models

In [29]:
models = (LogisticRegression(**log_params, max_iter = 1000).fit(X_train, y_train),
          SVC(**svm_params).fit(X_train, y_train),
          DecisionTreeClassifier(**tree_params).fit(X_train, y_train),
          RandomForestClassifier(**rf_params).fit(X_train, y_train),
          GradientBoostingClassifier(**gbt_params).fit(X_train, y_train),
          xgb.XGBClassifier(**xgb_params).fit(X_train, y_train),
          CatBoostClassifier(**cat_params).fit(X_train, y_train)
)

In [31]:
pickle.dump(models, open('./sav/model_Hstack.sav', 'wb'))